In [1]:
import numpy as np
import pandas as pd

import conv_model as conv
import tensorflow as tf

In [2]:
#### User input ####

## path of the train and test data
train_data = "../data/digit-recognizer/train.csv"
test_data = "../data/digit-recognizer/test.csv"

In [3]:
## read data into Dataframe
df_train = pd.read_csv(train_data)
df_train.info()

df_test = pd.read_csv(test_data)
df_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 42000 entries, 0 to 41999
Columns: 785 entries, label to pixel783
dtypes: int64(785)
memory usage: 251.5 MB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 28000 entries, 0 to 27999
Columns: 784 entries, pixel0 to pixel783
dtypes: int64(784)
memory usage: 167.5 MB


In [4]:
## get the first 400 rows of the data
#df_train = df_train.iloc[:400]
#print(len(df_train.columns))

In [5]:
## function to create input images
def create_input_images(df, start = 1):
    im_arr = []
    for row in range(0, len(df.index)):
        ## get the image values
        im_flat = np.array(df.iloc[row, start:])
        
        ## create the image frame
        image = np.reshape(im_flat, (28, 28, 1))
    
        im_arr.append(image)

    return np.array(im_arr, dtype = np.float32)

In [6]:
## create the arrays of input images
im_arr = create_input_images(df_train)
im_arr_test = create_input_images(df_test, start = 0)

In [7]:
#model1 = conv.ConvModel(conv_layer = 'Conv2D')
model1 = conv.ConvModel(conv_layer = 'TSConv')

In [8]:
from tensorflow.keras.losses import CategoricalCrossentropy

In [9]:
## compile the model
model1.compile(optimizer = 'adam', loss = CategoricalCrossentropy(), metrics = ["accuracy"])

In [10]:
## create training input and one_hot encode
Y = df_train["label"].to_numpy()
Y_oh = tf.one_hot(Y, 10)
im_arr.shape

(42000, 28, 28, 1)

In [11]:
## fit the model (currently just testing the design, no final run yet)
model1.fit(im_arr, Y_oh, batch_size = 32, epochs = 10)

Epoch 1/10


2024-04-09 16:10:39.752182: W tensorflow/tsl/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz


(None, 3136)
(None, 3136)
1313/1313 [==============================] - 53s 39ms/step - loss: 0.2377 - accuracy: 0.9269
Epoch 2/10
1313/1313 [==============================] - 56s 43ms/step - loss: 0.0927 - accuracy: 0.9714
Epoch 3/10
1313/1313 [==============================] - 58s 44ms/step - loss: 0.0680 - accuracy: 0.9791
Epoch 4/10
1313/1313 [==============================] - 61s 46ms/step - loss: 0.0561 - accuracy: 0.9818
Epoch 5/10
1313/1313 [==============================] - 64s 49ms/step - loss: 0.0468 - accuracy: 0.9851
Epoch 6/10
1313/1313 [==============================] - 66s 51ms/step - loss: 0.0420 - accuracy: 0.9871
Epoch 7/10
1313/1313 [==============================] - 66s 50ms/step - loss: 0.0370 - accuracy: 0.9881
Epoch 8/10
1313/1313 [==============================] - 66s 50ms/step - loss: 0.0341 - accuracy: 0.9888
Epoch 9/10
1313/1313 [==============================] - 67s 51ms/step - loss: 0.0294 - accuracy: 0.9908
Epoch 10/10
1313/1313 [==========================

In [12]:
model1.summary()

Model: "conv_model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 28, 28, 1)]       0         
                                                                 
 dropout (Dropout)           multiple                  0         
                                                                 
 conv_block (ConvBlock)      multiple                  232       
                                                                 
 db (DB)                     multiple                  400       
                                                                 
 conv_block_2 (ConvBlock)    multiple                  1184      
                                                                 
 db_1 (DB)                   multiple                  1568      
                                                                 
 conv_block_4 (ConvBlock)    multiple                  1

In [13]:
## predict for the test data
preds = model1.predict(im_arr_test)

(32, 3136)
875/875 [==============================] - 15s 17ms/step


In [14]:
## add test predictions to the test dataframe
#print(preds.shape)
#print(len(np.argmax(preds, axis = 1)))
#print(len(np.arange(1, len(df_test.index)+1)))
#print(df_test.columns)

df_test["Label"] = np.argmax(preds, axis = 1)
df_test["ImageId"] = np.arange(1, len(df_test.index)+1)

In [16]:
## path of the test predictions
test_pred_path = "../data/digit-recognizer/test_preds4.csv"

## write results to csv file
#to_write = ["ImageId", "Label"]
#df_test.to_csv(test_pred_path, columns = to_write, index = False)